In [ ]:
import torch
import torch.nn.functional as F
import torchvision.models as models
import matplotlib.pyplot as plt

from common_visualize import *
from utils import *
from cam.scorecam import *
from visualize import *

from torchvision.transforms import ToTensor
import cv2
from PIL import Image


In [ ]:
imagePath = []
targets = [58,222,332,417,429,429,525,549,683,889,990]

untrained = models.vgg16(pretrained=False)
untrained.eval()

features_ls = [30,28,26,24,21,19,17,14,12,10,7,5,2,0]

In [ ]:
for i, path in enumerate(imagePath):
  img = cv2.imread(path)

  tts = ToTensor()
  input_tensor = tts(img).unsqueeze(0)

  model = models.vgg16(pretrained=True)
  model.eval()
  trained_model_dict = dict(type='vgg16', arch=model, layer_name='features_29',input_size=(224, 224))
  trained_scorecam = ScoreCAM(trained_model_dict)

  for layer in features_ls:
    if layer == 28:
      model.classifier = untrained.classifier
    if layer < 30:
      model.features[layer] = untrained.features[layer]
      trained_scorecam = ScoreCAM(trained_model_dict)

    if torch.cuda.is_available():
      input_tensor = input_tensor.cuda()
    else:
      input_tensor = input_tensor.cpu()
    predicted_class = trained(input_tensor).max(1)[-1]

    scorecam_map = trained_scorecam(input_tensor)
    scorecam_map = scorecam_map.cpu().squeeze()

    save_path = ''
    save_result(scorecam_map, save_path)

compute SSIM score

In [ ]:
base_path = ''

ssim_ls = compute_ssim(imagePath, base_path)


In [ ]:
plot_ssim(ssim_ls, 'ScoreCAM', '')